In [1]:
!pip install -q ultralytics albumentations --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.4 MB/s eta 0:00:0000:0100:01


# **Copy dataset to working directory**

In [2]:
!cp -r "/kaggle/input/waste-detection/waste detection data/" "/kaggle/working/dataset"

# Generating yaml config file 

In [3]:
from ruamel.yaml import YAML
import os

ROOT_TRAINING_FOLDER = "/kaggle/working/dataset"
# Correct the paths in the data dictionary
data = {
    'train': os.path.join(ROOT_TRAINING_FOLDER, 'train', 'images'),
    'val': os.path.join(ROOT_TRAINING_FOLDER, 'valid', 'images'),
    'nc': 7,
    'names': ['Aluminium', 'Glass', 'Tag', 'cardboard', 'rigid_plastic', 'soft_plastic', 'wood']
}

yaml = YAML()
with open('data.yaml', 'w') as f:
    yaml.dump(data, f)

# Load dataset and create yaml

In [4]:
%%writefile /kaggle/working/data.yaml
# ============================================================
# YOLOv12 Dataset Configuration for Instance Segmentation
# ============================================================

train: /kaggle/working/dataset/train
val: /kaggle/working/dataset/valid
test: /kaggle/working/dataset/test

# Number of classes
nc: 7

# Class names
names: ['Aluminium','Glass','Tag','cardboard ', ' rigid_plastic','soft_plastic','wood']

Overwriting /kaggle/working/data.yaml


# Check and verify dataset

In [5]:
from ultralytics.data.utils import check_det_dataset

check_det_dataset('/kaggle/working/data.yaml')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


{'train': '/kaggle/working/dataset/train',
 'val': '/kaggle/working/dataset/valid',
 'test': '/kaggle/working/dataset/test',
 'nc': 7,
 'names': {0: 'Aluminium',
  1: 'Glass',
  2: 'Tag',
  3: 'cardboard ',
  4: ' rigid_plastic',
  5: 'soft_plastic',
  6: 'wood'},
 'yaml_file': '/kaggle/working/data.yaml',
 'channels': 3,
 'path': PosixPath('/kaggle/working')}

In [6]:
from ultralytics import YOLO
import os, glob, random, shutil, cv2
import numpy as np
import torch
import albumentations as A
from tqdm import tqdm

In [7]:
BASE_DATASET = "/kaggle/working/dataset"  # your dataset root
DATA_YAML    = "/kaggle/working/data.yaml"          # your baseline yaml (already exists)

LABELED_PATH   = "/kaggle/working/ts_labeled"
UNLABELED_PATH = "/kaggle/working/ts_unlabeled"
PSEUDO_PATH    = "/kaggle/working/ts_pseudo"
MERGED_PATH    = "/kaggle/working/ts_merged"

os.makedirs(f"{LABELED_PATH}/images", exist_ok=True)
os.makedirs(f"{LABELED_PATH}/labels", exist_ok=True)
os.makedirs(f"{UNLABELED_PATH}/images", exist_ok=True)
os.makedirs(f"{UNLABELED_PATH}/labels", exist_ok=True)
os.makedirs(f"{PSEUDO_PATH}/images", exist_ok=True)
os.makedirs(f"{PSEUDO_PATH}/labels", exist_ok=True)
os.makedirs(f"{MERGED_PATH}/images", exist_ok=True)
os.makedirs(f"{MERGED_PATH}/labels", exist_ok=True)

NUM_CLASSES = 7
CLASS_NAMES = ['Aluminium','Glass','Tag','cardboard ', ' rigid_plastic','soft_plastic','wood']

LABELED_FRACTION = 0.2   # 20% labeled, 80% unlabeled
TEACHER_EPOCHS   = 10
STUDENT_EPOCHS   = 20
IMGSZ            = 640
BATCH            = 8
DEVICE           = 0                # GPU id or 'cpu'
PSEUDO_CONF_TH   = 0.6              # confidence threshold for accepting teacher preds
MAX_PSEUDO       = None             # set e.g. 500 to limit pseudo-labeling for speed

# ============================================================
# 1) Split dataset into labeled/unlabeled (keeping labels with labeled split)
# ============================================================
all_train_imgs = [p for p in glob.glob(f"{BASE_DATASET}/train/images/*.*")
                  if p.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"))]
random.shuffle(all_train_imgs)

split_idx = int(len(all_train_imgs) * LABELED_FRACTION)
labeled_imgs   = all_train_imgs[:split_idx]
unlabeled_imgs = all_train_imgs[split_idx:]

def safe_copy_pairs(image_paths, dst_root):
    """Copy images and their YOLO label .txt (if exists) to dst_root/images, dst_root/labels."""
    os.makedirs(f"{dst_root}/images", exist_ok=True)
    os.makedirs(f"{dst_root}/labels", exist_ok=True)
    for img in image_paths:
        base = os.path.splitext(os.path.basename(img))[0]
        lbl  = img.replace("/images/", "/labels/").rsplit(".", 1)[0] + ".txt"
        shutil.copy(img, f"{dst_root}/images/{os.path.basename(img)}")
        if os.path.exists(lbl):
            shutil.copy(lbl, f"{dst_root}/labels/{base}.txt")

safe_copy_pairs(labeled_imgs,   LABELED_PATH)
safe_copy_pairs(unlabeled_imgs, UNLABELED_PATH)

print(f"✅ Split → Labeled: {len(labeled_imgs)} | Unlabeled: {len(unlabeled_imgs)}")

# (Optional) quick integrity echo
lab_img_names = {os.path.splitext(f)[0] for f in os.listdir(f"{LABELED_PATH}/images")}
lab_lbl_names = {os.path.splitext(f)[0] for f in os.listdir(f"{LABELED_PATH}/labels")}
print(f"🧩 Integrity (labeled): images={len(lab_img_names)} labels={len(lab_lbl_names)}  missing_labels={len(lab_img_names - lab_lbl_names)}")

✅ Split → Labeled: 3214 | Unlabeled: 12859
🧩 Integrity (labeled): images=3214 labels=1591  missing_labels=1623


In [8]:
# ============================================================
# 2) Build labeled YAML for Teacher training
# ============================================================
import yaml as pyyaml

DATA_YAML_LABELED = "/kaggle/working/ts_labeled.yaml"

with open(DATA_YAML_LABELED, "w") as f:
    pyyaml.safe_dump({
        "train": f"{LABELED_PATH}/images",
        "val": f"{BASE_DATASET}/valid/images",
        "test": f"{BASE_DATASET}/test/images",
        "nc": NUM_CLASSES,
        "names": CLASS_NAMES
    }, f, sort_keys=False)

print(open(DATA_YAML_LABELED).read())


train: /kaggle/working/ts_labeled/images
val: /kaggle/working/dataset/valid/images
test: /kaggle/working/dataset/test/images
nc: 7
names:
- Aluminium
- Glass
- Tag
- 'cardboard '
- ' rigid_plastic'
- soft_plastic
- wood



In [9]:
import os

# REMOVE CUDA COMPLETELY (not empty string)
if "CUDA_VISIBLE_DEVICES" in os.environ:
    del os.environ["CUDA_VISIBLE_DEVICES"]

print("CUDA_VISIBLE_DEVICES =", os.environ.get("CUDA_VISIBLE_DEVICES"))


CUDA_VISIBLE_DEVICES = None


In [10]:
# ============================================================
# 3) Train Teacher on the labeled subset
# =====================================
import torch
from ultralytics import YOLO

print("CUDA available:", torch.cuda.is_available())

teacher = YOLO("yolo12n-seg.yaml").load("yolo12n.pt")

teacher_results = teacher.train(
    data=DATA_YAML_LABELED,
    epochs=TEACHER_EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device="cpu",     # CPU only
    amp=False,        # AMP OFF
    project="/kaggle/working/ts_runs",
    name="teacher_yolo12_seg",
    exist_ok=True,
    patience=10
)

teacher_best = f"{teacher_results.save_dir}/weights/best.pt"
teacher = YOLO(teacher_best)

print("✅ Training started successfully")


CUDA available: False
Transferred 691/753 items from pretrained weights
Ultralytics 8.3.239 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/ts_labeled.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n-seg.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=teacher_yolo12_seg, nbs=64, nms=False, opset=None, optim

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       1270       4483      0.581      0.511      0.523      0.344      0.564      0.486      0.481      0.261
             Aluminium        151        154      0.955      0.331      0.543      0.389      0.981      0.331      0.543      0.395
                 Glass        104        116      0.686      0.659      0.756      0.542      0.691      0.621      0.681      0.283
                   Tag        267        301      0.451      0.505      0.459      0.261       0.45      0.488      0.446      0.227
            cardboard         566       2312      0.576      0.391      0.429       0.22      0.542      0.345      0.361       0.14
         rigid_plastic        609        682       0.66      0.632      0.663      0.493      0.626      0.591        0.6       0.32
          soft_plastic        429        895      0.526      0.145      0.235      0.101      0.434      0.109      0.155       0.05
                  wood         23         23      0.214      0.913   

In [11]:
# ============================================================
# 4) Generate pseudo-labels on unlabeled images with Teacher
# ============================================================
to_label = unlabeled_imgs if MAX_PSEUDO is None else unlabeled_imgs[:MAX_PSEUDO]
print(f"📝 Pseudo-labeling {len(to_label)} unlabeled images (conf ≥ {PSEUDO_CONF_TH})")

def write_seg_label_file(out_txt, classes, masks_xy, orig_shape):
    """
    Write YOLO segmentation polygons to txt:
    <cls> x1 y1 x2 y2 ... (normalized)
    """
    oh, ow = orig_shape  # H, W
    with open(out_txt, "w") as f:
        for cls_id, poly in zip(classes, masks_xy):
            if poly is None or len(poly) < 3:
                continue
            # normalize polygon
            poly = np.array(poly, dtype=np.float32)
            poly[:, 0] = np.clip(poly[:, 0] / ow, 0.0, 1.0)
            poly[:, 1] = np.clip(poly[:, 1] / oh, 0.0, 1.0)
            coords = " ".join([f"{x:.6f}" for x in poly.reshape(-1)])
            f.write(f"{int(cls_id)} {coords}\n")

for img_path in tqdm(to_label, desc="Generating pseudo-labels"):
    # Single-image predict (faster if batched, but simple & robust here)
    results = teacher.predict(source=img_path, imgsz=IMGSZ, conf=PSEUDO_CONF_TH, save=False, verbose=False)
    if not results:
        continue
    r = results[0]
    if r.masks is None or len(r.boxes) == 0:
        continue

    # Prepare output files
    base = os.path.splitext(os.path.basename(img_path))[0]
    out_img = f"{PSEUDO_PATH}/images/{os.path.basename(img_path)}"
    out_lbl = f"{PSEUDO_PATH}/labels/{base}.txt"

    # filter instances by conf
    confs = r.boxes.conf.cpu().numpy()
    keep_idx = np.where(confs >= PSEUDO_CONF_TH)[0]
    if keep_idx.size == 0:
        continue

    # Extract classes & polygons for kept instances
    classes = []
    masks_xy = []
    for i in keep_idx:
        # masks.xy is a list of polygons; some instances may have multiple polygons (holes). Use largest.
        if r.masks is None or r.masks.xy is None:
            continue
        polys = r.masks.xy[i]  # list of (N_i, 2) ndarray or single (N,2) depending on Ultralytics version
        if isinstance(polys, list):
            # choose largest polygon by area
            areas = [cv2.contourArea(np.array(p, dtype=np.float32)) for p in polys if len(p) >= 3]
            if len(areas) == 0:
                continue
            poly = polys[int(np.argmax(areas))]
        else:
            poly = polys
        if poly is None or len(poly) < 3:
            continue
        classes.append(int(r.boxes.cls[i].item()) if hasattr(r.boxes, "cls") else 0)
        masks_xy.append(poly)

    if len(masks_xy) == 0:
        continue

    # Save label + image copy
    h, w = r.orig_shape  # (H, W)
    write_seg_label_file(out_lbl, classes, masks_xy, (h, w))
    shutil.copy(img_path, out_img)

print("✅ Pseudo-labels done.")

📝 Pseudo-labeling 12859 unlabeled images (conf ≥ 0.6)


Generating pseudo-labels: 100%|██████████| 12859/12859 [1:00:35<00:00,  3.54it/s]

✅ Pseudo-labels done.


In [12]:
import yaml
import glob, shutil, os

# ============================================================
# 5) Merge labeled and pseudo-labeled into one training set
# ============================================================
def merge_dir(src, dst):
    os.makedirs(dst, exist_ok=True)
    for f in glob.glob(f"{src}/*"):
        shutil.copy(f, f"{dst}/{os.path.basename(f)}")

merge_dir(f"{LABELED_PATH}/images", f"{MERGED_PATH}/images")
merge_dir(f"{LABELED_PATH}/labels", f"{MERGED_PATH}/labels")
merge_dir(f"{PSEUDO_PATH}/images", f"{MERGED_PATH}/images")
merge_dir(f"{PSEUDO_PATH}/labels", f"{MERGED_PATH}/labels")

DATA_YAML_MERGED = "/kaggle/working/ts_merged.yaml"

with open(DATA_YAML_MERGED, "w") as f:
    yaml.safe_dump(
        {
            "train": f"{MERGED_PATH}/images",
            "val": f"{BASE_DATASET}/valid/images",
            "test": f"{BASE_DATASET}/test/images",
            "nc": NUM_CLASSES,
            "names": CLASS_NAMES,
        },
        f,
        sort_keys=False,
    )

# print statement 
print(open(DATA_YAML_MERGED).read())



train: /kaggle/working/ts_merged/images
val: /kaggle/working/dataset/valid/images
test: /kaggle/working/dataset/test/images
nc: 7
names:
- Aluminium
- Glass
- Tag
- 'cardboard '
- ' rigid_plastic'
- soft_plastic
- wood



In [ ]:
import torch
from ultralytics import YOLO

# ============================================================
# 6) Train Student on merged dataset
# ============================================================

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

student = YOLO("yolo12n-seg.yaml").load("yolo12n.pt")

student_results = student.train(
    data=DATA_YAML_MERGED,
    epochs=STUDENT_EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=DEVICE,               # ✅ no invalid CUDA request
    project="/kaggle/working/ts_runs",
    name="student_yolo12_seg",
    exist_ok=True,
    patience=10,
    amp=True
)

student_best = f"{student_results.save_dir}/weights/best.pt"
print("✅ Student best:", student_best)
